In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
# Load environment variables in a file called .env
#You have to use your API Key or you can check the same project with the Ollama

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found!!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end. Please remove them")
else:
    print("API key found and everything is fine so far!")


API key found and looks good so far!


In [3]:
openai = OpenAI()

In [4]:
#Quick check with simple example

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello! Welcome, and it's great to hear from you! How can I assist you today?


In [7]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

# A class to represent a Webpage
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [8]:
#Let's check the one website for an example
ex = Website("https://vg247.com")
print(ex.title)
print(ex.text)

VG247
Skip to main content
Monopoly Go Dice Links
Roblox Hunt code
Volleyball Legends codes
Fragpunk Tier List
Monster Hunter Wilds Guide
Roblox game codes
Basketball Showdown Codes
News
Reviews
Articles
Guides
Baldur's Gate 3
Diablo IV
Fallout 4
Elden Ring
Black Myth: Wukong
Deals
Sign in / Create account
Support us
Buy eShop codes
Home
For you
Build-A-Battlefield
I’ve watched enough leaked Battlefield Labs gameplay to convince me that Battlefield is finally getting this one fundamental thing right, at least
DICE and all the other teams currently working on the next Battlefield are, finally, building on what came before.
Sherif Saed
3 days ago
1
Temple of the mind
Want to know how Warframe got its own bizarre Sci-Fi David Bowie? Here's how Temple came to be, from a sketch to your Sortie
"This one in particular was one of the most self-indulgent, coolest set of things I could put into a Warframe"
Connor Makar
3 days ago
0
Barging Onto Market
A Nintendo Switch 2 may set you back $400 or

In [9]:
# Let's define our system prompt

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [10]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [11]:
#Function check

print(user_prompt_for(ex))

You are looking at a website titled VG247
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to main content
Monopoly Go Dice Links
Roblox Hunt code
Volleyball Legends codes
Fragpunk Tier List
Monster Hunter Wilds Guide
Roblox game codes
Basketball Showdown Codes
News
Reviews
Articles
Guides
Baldur's Gate 3
Diablo IV
Fallout 4
Elden Ring
Black Myth: Wukong
Deals
Sign in / Create account
Support us
Buy eShop codes
Home
For you
Build-A-Battlefield
I’ve watched enough leaked Battlefield Labs gameplay to convince me that Battlefield is finally getting this one fundamental thing right, at least
DICE and all the other teams currently working on the next Battlefield are, finally, building on what came before.
Sherif Saed
3 days ago
1
Temple of the mind
Want to know how Warframe got its own bizarre Sci-Fi David Bowie? Here's how Temple came to be, from a sketch to your Sor

# Let's ask to OpenAI

In [13]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
# And now: call the OpenAI API

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [15]:
summarize("https://geekozmos.com")

'# Geekozmos Summary\n\nGeekozmos is a comprehensive platform dedicated to tech, gaming, and entertainment news, reviews, and updates. Launched on August 1st, 2023, it serves as a centralized resource for enthusiasts in these fields.\n\n## Latest News & Announcements\n\n### Tech Updates:\n- **Battery Health Feature in Android 15**: Anticipation grows for a new battery health feature in Android 15, aimed at providing users with insights on device performance.\n- **Apple Watch Sales Halt**: Apple has paused sales of the Apple Watch Series 9 and Ultra 2 due to a patent dispute concerning blood oxygen sensing technology.\n- **iOS 17 Bug**: A bug in iOS 17 is reportedly causing the new iPhone 15 models to freeze.\n\n### Gaming Insights:\n- **PS4 Compatibility**: PS5 is confirmed to be backward compatible with most PS4 games.\n- **Death Stranding 2 Release**: A new trailer for "Death Stranding 2: On The Beach" has been released, with the game slated for 2025.\n- **System Requirements for Pri

In [16]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [17]:
display_summary("https://geekozmos.com")

# Summary of Geekozmos

**Geekozmos** is a comprehensive online platform that focuses on tech, gaming, and entertainment news. Established in August 2023, it aims to be a go-to resource for enthusiasts in these fields, providing the latest updates, insightful reviews, and captivating content related to movies, TV shows, comics, and video games.

## Latest News

### Tech
- **Battery Health Feature May Come With Android 15**: Android 15 is expected to introduce a feature that will give users insights into their device's battery performance.
- **Apple Halts Sales of Apple Watch Series 9 and Ultra 2**: Apple has paused sales of these models due to a patent dispute with Masimo.
- **iOS 17 Bug Causing New iPhone 15 Models to Freeze**: There is a reported bug affecting the new iPhone 15, leading to device freezes.

### Gaming
- **Can You Play PS4 Games on PS5?**: The PS5 supports the vast majority of PS4 games thanks to backward compatibility.
- **Death Stranding 2: On The Beach Set for 2025 Release**: A new trailer has been released for this sequel, which will be exclusive to PlayStation 5.
- **Prince of Persia: The Lost Crown System Requirements Announced**: Hardware requirements for this upcoming game have been detailed.
- **Insomniac Games’ Next Title Will Be Marvel’s Venom**: Leaked files indicate that Insomniac Games is developing a new game featuring the Marvel character Venom.

### Entertainment
- **Deadpool 3 Trailer Released**: Fans have received the first glimpse of the upcoming Deadpool movie featuring Wolverine.
- **Matthew Perry Passes Away at 54**: The beloved actor known for his role in "Friends" has died.
- **Highlighted Career of Léa Seydoux**: A retrospective on the notable movies starring the actress has been published.

## Reviews
The site features several reviews for games, including:
- **Anomaly Agent**: 7/10
- **Ghost of Tsushima**: 8/10
- **Marvel's Spider-Man**: 8.5/10

Geekozmos provides a well-rounded experience for users interested in the latest happenings in technology, gaming, and entertainment, making it a valuable resource for fans and enthusiasts alike.

In [ ]:
#You can try more. For example:

display_summary("https://cnn.com")